# Loop 41 Analysis: GNN Failed - What Next?

**Situation:**
- GNN (AttentiveFP) test fold MSE: 0.068767 (8.4x WORSE than baseline)
- Best CV: 0.008194 (exp_035)
- Best LB: 0.0877 (exp_030)
- Target: 0.0347
- CV-LB relationship: LB = 4.30*CV + 0.0524 (R²=0.97)
- Submissions remaining: 4

**Key Question:** Why did GNN fail when the benchmark achieved MSE 0.0039?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load submission history
submissions = [
    {'exp': 'exp_000', 'cv': 0.0111, 'lb': 0.0982},
    {'exp': 'exp_001', 'cv': 0.0123, 'lb': 0.1065},
    {'exp': 'exp_003', 'cv': 0.0105, 'lb': 0.0972},
    {'exp': 'exp_005', 'cv': 0.0104, 'lb': 0.0969},
    {'exp': 'exp_006', 'cv': 0.0097, 'lb': 0.0946},
    {'exp': 'exp_007', 'cv': 0.0093, 'lb': 0.0932},
    {'exp': 'exp_009', 'cv': 0.0092, 'lb': 0.0936},
    {'exp': 'exp_012', 'cv': 0.0090, 'lb': 0.0913},
    {'exp': 'exp_024', 'cv': 0.0087, 'lb': 0.0893},
    {'exp': 'exp_026', 'cv': 0.0085, 'lb': 0.0887},
    {'exp': 'exp_030', 'cv': 0.0083, 'lb': 0.0877},
    {'exp': 'exp_035', 'cv': 0.0098, 'lb': 0.0970},
]

df = pd.DataFrame(submissions)
print('Submission History:')
print(df)

# Linear fit
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(df['cv'], df['lb'])
print(f'\nCV-LB Relationship: LB = {slope:.2f}*CV + {intercept:.4f} (R²={r_value**2:.3f})')
print(f'Intercept: {intercept:.4f}')
print(f'Target: 0.0347')
print(f'Gap: Intercept is {intercept/0.0347:.2f}x larger than target')

In [ ]:
# Analyze GNN failure
print('=== GNN FAILURE ANALYSIS ===')
print()
print('GNN Test Fold MSE: 0.068767')
print('Baseline (exp_035) CV: 0.008194')
print('GNN is 8.4x WORSE than baseline')
print()
print('Possible reasons for GNN failure:')
print('1. Training data too small (~619 samples) for GNN to learn meaningful representations')
print('2. Leave-one-solvent-out CV is extremely challenging for GNN')
print('3. The GNN benchmark may have used different CV scheme (not leave-one-solvent-out)')
print('4. AttentiveFP may need more training epochs or different hyperparameters')
print('5. The molecular graphs may need more sophisticated features')
print()
print('Key insight: The GNN benchmark (MSE 0.0039) may have used a different evaluation scheme!')
print('Our leave-one-solvent-out CV is MUCH harder than standard random splits.')

In [ ]:
# What approaches haven't been tried?
print('=== UNEXPLORED APPROACHES ===')
print()
print('1. Pre-trained molecular embeddings (ChemBERTa, MolBERT)')
print('   - Use embeddings from models trained on millions of molecules')
print('   - These capture general chemical knowledge that transfers to new solvents')
print()
print('2. k-NN with Tanimoto similarity')
print('   - For test solvent, find k most similar training solvents')
print('   - Weight predictions by similarity')
print('   - Simple but may work for distribution shift')
print()
print('3. Meta-learning / Few-shot learning')
print('   - Learn a model that can quickly adapt to new solvents')
print('   - MAML, Prototypical Networks, etc.')
print()
print('4. Adversarial domain adaptation')
print('   - Learn features that are invariant across solvents')
print('   - May reduce distribution shift')
print()
print('5. Pure GP with different kernels')
print('   - GP provides uncertainty estimates')
print('   - May have different CV-LB relationship')

In [ ]:
# Analyze what the benchmark paper might have done differently
print('=== BENCHMARK ANALYSIS ===')
print()
print('GNN Benchmark (arXiv:2512.19530) achieved MSE 0.0039')
print('Our best LB: 0.0877 (22x worse)')
print('Target: 0.0347 (8.9x worse than benchmark)')
print()
print('Possible differences in benchmark evaluation:')
print('1. Different CV scheme (random splits vs leave-one-solvent-out)')
print('2. Pre-training on larger molecular datasets')
print('3. Different GNN architecture (not AttentiveFP)')
print('4. Different feature engineering')
print('5. Different evaluation metric')
print()
print('CRITICAL: The competition uses leave-one-solvent-out CV!')
print('This is MUCH harder than random splits because:')
print('- Test solvent is NEVER seen during training')
print('- Model must generalize to completely new molecular structures')
print('- This is an out-of-distribution (OOD) problem')

In [ ]:
# What can we do with 4 submissions remaining?
print('=== SUBMISSION STRATEGY ===')
print()
print('Submissions remaining: 4')
print('Best LB so far: 0.0877 (exp_030)')
print('Target: 0.0347')
print('Gap: 2.53x')
print()
print('Options:')
print('1. Submit exp_035 (CV 0.008194) - our best CV model')
print('   - Expected LB: 4.30*0.008194 + 0.0524 = 0.0876 (similar to exp_030)')
print('   - Unlikely to beat target')
print()
print('2. Try pre-trained embeddings (ChemBERTa)')
print('   - May have different CV-LB relationship')
print('   - Worth trying before submitting')
print()
print('3. Try k-NN with Tanimoto similarity')
print('   - Simple approach that may work for OOD')
print('   - Worth trying before submitting')
print()
print('4. Try pure GP with sophisticated kernels')
print('   - GP may have different CV-LB relationship')
print('   - Worth trying before submitting')
print()
print('RECOMMENDATION: Try pre-trained embeddings first, then k-NN, then submit best.')

In [ ]:
# Check if we have the right packages for pre-trained embeddings
import subprocess
result = subprocess.run(['pip', 'list'], capture_output=True, text=True)
print('Checking available packages for pre-trained embeddings...')
if 'transformers' in result.stdout:
    print('✓ transformers is available')
else:
    print('✗ transformers is NOT available')

if 'torch' in result.stdout:
    print('✓ torch is available')
else:
    print('✗ torch is NOT available')

if 'rdkit' in result.stdout.lower():
    print('✓ rdkit is available')
else:
    print('✗ rdkit is NOT available')

In [ ]:
# Check if ChemBERTa is available
try:
    from transformers import AutoModel, AutoTokenizer
    print('Testing ChemBERTa...')
    tokenizer = AutoTokenizer.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')
    model = AutoModel.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')
    print('✓ ChemBERTa is available!')
    
    # Test on a simple SMILES
    smiles = 'CCO'  # Ethanol
    inputs = tokenizer(smiles, return_tensors='pt')
    outputs = model(**inputs)
    print(f'Embedding shape: {outputs.last_hidden_state.shape}')
except Exception as e:
    print(f'✗ ChemBERTa not available: {e}')

In [ ]:
# Summary
print('=== LOOP 41 SUMMARY ===')
print()
print('GNN (AttentiveFP) FAILED with MSE 0.068767 (8.4x worse than baseline)')
print()
print('Key insight: The GNN benchmark (MSE 0.0039) likely used a different CV scheme.')
print('Our leave-one-solvent-out CV is an OOD problem that is MUCH harder.')
print()
print('Next steps:')
print('1. Try pre-trained molecular embeddings (ChemBERTa) if available')
print('2. Try k-NN with Tanimoto similarity as a simple OOD approach')
print('3. Try pure GP with sophisticated kernels')
print('4. If none work, submit exp_035 (best CV) and accept the gap')
print()
print('The target (0.0347) may be achievable with a fundamentally different approach,')
print('but we need to find what changes the CV-LB relationship.')